In [1]:
!pip install albumentations
!pip install torch
!pip install segmentation-models-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.9 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=41a53b36a1228a47115d5508c11b4ad2f684f5922dea2dc32ef1017bf0ee777d
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60945 sha256=2ab164de6bdeea3717e1977886a2227d8c6b68427e2d53e3b3780dc6446b6f03
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-pytorch pretrainedmodels


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
ROOT = '/content/drive/My Drive/Western_blot'
SEC = '/content/drive/My Drive/Augmented_Western_blot'

In [8]:
import os
import numpy as np
from PIL import Image
def load_images(root_folder):
    # Array to store loaded images
    images = []
    for filename in os.listdir(root_folder):
        filepath = os.path.join(root_folder, filename)
        # Open the image file using PIL and convert it to a numpy array
        image = np.array(Image.open(filepath))
         # Append the numpy array representing the image to the list of images
        images.append(image)
    return images

In [9]:
images = load_images(ROOT)

In [10]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from PIL import Image
import segmentation_models_pytorch as smp

class CustomImageDataset(Dataset):
    def __init__(self, image_list, labels, transform=None):
        self.image_list = image_list
        self.transform = transform
        self.labels = labels

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx, label):
        print(idx)
        # Get the path of the image at the specified index
        image_path = self.image_list[idx]
        # Open the image and convert it to grayscale
        image = Image.open(image_path).convert('L')
        # Transform image
        if self.transform:
            image = self.transform(image)
        # Get the label corresponding to the image
        label = self.transform(image)
        return image, label

transform = transforms.Compose([
    # Resize image
    transforms.Resize((224, 224)),
    # Convert image to PyTorch tensor
    transforms.ToTensor(),
    # Normalize image
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
# Create an instance of the CustomImageDataset class with the transformation
dataset = CustomImageDataset(images, transform=transform)



TypeError: CustomImageDataset.__init__() missing 1 required positional argument: 'labels'

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
# Create a DataLoder to load data in batches
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

# resnet = models.resnet50(pretrained=True)

# for param in resnet.parameters():
#     param.requires_grad = False

# Define number of classes in classification task
num_classes = 5

# class CustomClassifier(torch.nn.Module):
#     def __init__(self, in_features, num_classes):
#         super(CustomClassifier, self).__init__()
#         self.fc1 = torch.nn.Linear(in_features, 512)
#         self.fc2 = torch.nn.Linear(512, num_classes)
#         self.relu = torch.nn.ReLU()
#         self.dropout = torch.nn.Dropout(0.5)

#     def forward(self, x):
#         x = self.relu(self.fc1(x))
#         x = self.dropout(x)
#         x = self.fc2(x)
#         return x

# resnet.fc = CustomClassifier(resnet.fc.in_features, num_classes)

# for param in resnet.fc.parameters():
#     param.requires_grad = True

# Parameters of UNet model architecture
model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=1,
    classes=num_classes,
)

# Freeze parameters of the ResNet encoder to prevent them from updating during training
for param in model.encoder.parameters():
    param.requires_grad = False

# Define loss function
criterion = torch.nn.CrossEntropyLoss()

# Define optimizer for updating model parameters
optimizer = torch.optim.Adam(resnet.parameters(), lr=0.001)

num_epochs = 10

# Iterate over each epoch
for epoch in range(num_epochs):
    # Iterate over each batch of data
    for images, labels in dataloader:
        # Reset gradients to 0
        optimizer.zero_grad()
        # Compute predicted outputs by passing inputs to model
        outputs = resnet(images)
        # Calculate loss
        loss = criterion(outputs, labels)
        # Compute gradient of loss in respect to model parameters
        loss.backward()
        # Update model parameters
        optimizer.step()
    # Prints epoch number and loss after each epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Save trained model's state dictionary to file
torch.save(model.state_dict(), 'model.pth')
